In [1]:
import pandas as pd
import gmaps
import requests

from config import g_key

In [2]:
city_data_df = pd.read_csv('weather_data/cities.csv')
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Cape Town,ZA,2021-06-15 17:53:32,-33.9258,18.4232,60.89,78,75,6.91
1,1,Mar Del Plata,AR,2021-06-15 17:53:32,-38.0023,-57.5575,53.01,74,20,11.99
2,2,Berdychiv,UA,2021-06-15 17:53:32,49.8993,28.6024,62.82,92,17,8.37
3,3,Esperance,AU,2021-06-15 17:53:33,-33.8667,121.9000,51.64,67,78,15.41
4,4,Pevek,RU,2021-06-15 17:53:33,69.7008,170.3133,33.57,93,98,6.42


In [3]:
city_data_df.dtypes # Working with gmaps dependency requires the lats and lngs to be int or floats

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

# Create a max temperature heatmap

In [4]:
# Configure gmaps to use API key
gmaps.configure(api_key = g_key)

### The general syntax for creating a heat map is as follows

Do not run code

In [6]:
# # 1. Assign the locations to an array of latitude and longitude pairs.
# locations = [latitude, longitude]
# # 2. Assign the weights variable to some values.
# temperatures = # an array of length equal to the locations array length
# # 3. Assign the figure variable to the gmaps.figure() attribute.
# fig = gmaps.figure()
# # 4. Assign the heatmap_layer variable to the heatmap_layer attribute and add in the locations.
# heatmap_layer = gmaps.heatmap_layer(locations, weights=temperatures)

# # 5. Add the heatmap layer.
# fig.add_layer(heatmap_layer)
# # 6. Call the figure to plot the data.
# fig

Code for doing it:

In [5]:
locations = city_data_df[['Lat','Lng']]

max_temp = city_data_df['Max Temp']

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

If we were to have a prob running the previous line of code with a InvalidWeightException Error, it would be beacuse of negative temperatures for some cities. In order to solve this problem... we could create a new array temps and use it as the weights in the heat_layer creation instead

In [6]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))
#temps
# And use temps instead of max_temp as weight of heat layer

Or code it directly in the gmaps.heatmap_layer() module using list comprehension as *weights=[max(temp,0) for temp in max_temp]*

# To adjust heatmap zoom, intensity, and point radius

In [7]:
# MAX TEMPERATURE HEAT MAP
locations = city_data_df[['Lat','Lng']]

max_temp = city_data_df['Max Temp']

fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5) # CHANGES MADE: center and zoom level
                                                       # center is the geographic center of earth in lat, long form
                                                       # we also added zoom so that only one world map was shown
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
                                max_intensity=300, point_radius=3)
                                # CHANGES: added dissipating, max intensity, and point radius
                                # dissipation is default true, max int makes each measure have a better gradient
                                # variance. pt raidus to make each measurerment radius larger
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

# To create a % Humidity Map

In [9]:
locations = city_data_df[['Lat','Lng']]
humidity = city_data_df['Humidity']

fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False,
                                max_intensity=300, point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

# Create a % Cloudiness Heatmap


In [10]:
locations = city_data_df[['Lat','Lng']]
clouds = city_data_df['Cloudiness']

fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False,
                                max_intensity=300, point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

# Create a Wind Speed Heatmap


In [11]:
locations = city_data_df[['Lat','Lng']]
wind = city_data_df['Wind Speed']

fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False,
                                max_intensity=300, point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

We are going to ask the customer to input the temperature ranges they want for their perfect vacation

In [12]:
min_temp = float(input('What is the minimum temperature you would like for your trip?'))
max_temp = float(input('What is the maximum temperature you would like for your trip?'))


What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


In [13]:
# Creating new dataframe with only cities applicable to the customer's need
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & \
                                        (city_data_df['Max Temp'] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
7,7,Zhanakorgan,KZ,2021-06-15 17:53:34,43.9098,67.2495,79.11,23,0,9.95
8,8,Buala,SB,2021-06-15 17:53:34,-8.1450,159.5921,79.81,83,92,7.61
10,10,Kloulklubed,PW,2021-06-15 17:53:34,7.0419,134.2556,82.53,72,97,4.50
14,14,Atuona,PF,2021-06-15 17:53:35,-9.8000,-139.0333,78.22,80,33,19.77
25,25,Bodden Town,KY,2021-06-15 17:49:45,19.2833,-81.2500,89.60,1,20,18.41
26,26,Nouakchott,MR,2021-06-15 17:52:30,18.0858,-15.9785,78.73,82,30,12.17
30,30,Kapaa,US,2021-06-15 17:53:40,22.0752,-159.3190,77.18,75,76,1.01
36,36,Butaritari,KI,2021-06-15 17:53:42,3.0707,172.7902,82.96,77,100,16.71
37,37,Puerto Leguizamo,CO,2021-06-15 17:53:42,-0.1934,-74.7819,77.74,82,98,1.90
38,38,Sulangan,PH,2021-06-15 17:53:42,11.1373,123.7228,84.22,73,94,1.30


In [14]:
# To check if there are any null values in the new df
preferred_cities_df.isnull().sum()

City_ID       0
City          0
Country       0
Date          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
dtype: int64

Now that we have the cities that our customers are interested in, we need to go further and see where they can stay. Which hotel? We are going to select one hotel form each city using google maps.

Once the customers have filtered the database (DataFrame) based on their temperature preferences, show them a heatmap for the maximum temperature for the filtered cities. In addition, create a marker for each city that will display the name of the city, country code, maximum temperature, and name of a nearby hotel within three miles of the coordinates when the marker is clicked.

In [15]:
# Creat a new df called hotel_df to store hotel names along with these columns
hotel_df = preferred_cities_df[['City','Country','Max Temp','Lat','Lng']].copy()
hotel_df['Hotel Name'] = ''
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
7,Zhanakorgan,KZ,79.11,43.9098,67.2495,
8,Buala,SB,79.81,-8.1450,159.5921,
10,Kloulklubed,PW,82.53,7.0419,134.2556,
14,Atuona,PF,78.22,-9.8000,-139.0333,
25,Bodden Town,KY,89.60,19.2833,-81.2500,
26,Nouakchott,MR,78.73,18.0858,-15.9785,
30,Kapaa,US,77.18,22.0752,-159.3190,
36,Butaritari,KI,82.96,3.0707,172.7902,
37,Puerto Leguizamo,CO,77.74,-0.1934,-74.7819,
38,Sulangan,PH,84.22,11.1373,123.7228,


Using the latitude and longitude and specific parameters, use the Google Places Nearby Search request to retrieve a hotel and add it to the Hotel Name column.

# Create API request to look for closest hotel

In [16]:
# Set parameters to search for a hotel
params = {
    'radius':5000,
    'type':'lodging',
    'key':g_key
}

In [18]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]

IndexError: list index out of range

We need to try again with a try except so that our code is not stopped prematurely. 

In [17]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [18]:
hotel_df.tail(10) # As we can see, there were cases where it didn't find any hotel

,City,Country,Max Temp,Lat,Lng,Hotel Name
552,Cagliari,IT,80.94,39.2450,9.0912,Holiday Inn Cagliari
555,Valverde Del Camino,ES,80.08,37.5751,-6.7543,HOTEL MONTEAROMA (ulamde hotelera S.L.)
556,Le Creusot,FR,84.42,46.8071,4.4163,Hotel La Petite Verrerie
558,Bartica,GY,77.32,6.4000,-58.6167,Relax Inn Hotel
559,Marzuq,YE,85.19,14.4000,46.4667,
562,Filadelfia,US,82.02,39.9523,-75.1638,"The Ritz-Carlton, Philadelphia"
565,Kampot,KH,80.15,10.6167,104.1833,Two Moons Hotel
567,Changde,CN,81.34,29.0464,111.6783,Kangshida Hotel
568,Tautira,PF,78.82,-17.7333,-149.1500,Pension Oaoa
573,Yaounde,CM,80.13,3.8667,11.5167,Hotel Mérina


# Last Task: Add pop-up markers with hotel info to a heatmap

In [19]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [20]:
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=3)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))